<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150 align="right">
<img src=images/Acme.jpeg width=100 align="left">

<a id =1></a>
## KMIP Secrets Engine


## Business Values:
 * allows Vault to act as a Key Management Interoperability Protocol (KMIP) server provider and handle the lifecycle of its KMIP managed objects. 
 * relatively short time to live (TTL), often for the applications lifespan.
 * eliminate sharing, unique certificate for each application
 
KMIP is a standardized protocol that allows services and applications to perform cryptographic operations without having to manage cryptographic material, otherwise known as managed objects, by delegating its storage and lifecycle to a key management server.

It delegates the key management task to an external provider via KMIP protocol.


### Vault Setup

### Prerequisites
* Running Vault instance (in dev mode) 


In [ ]:
vault -version

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
vault status

In [ ]:
vault login root

---
### Enable the KMIP Secret Engine


In [ ]:
vault secrets enable kmip

In [ ]:
vault write kmip/config listen_addrs=0.0.0.0:5696

**When KMIP is configured**
it generates and stores a key-pair and uses it to sign a root CA which is used to sign an intermediate CA.  The intermediate CA is used to sign the server TLS certificates used by the KMIP listener to identity itself during the TLS handshake.

In [ ]:
vault read kmip/config

**Default certificate type is `ec` Elliptical Curve**   
If you need RSA, specify the certificate type and bits when you configure the kmip server.

In [ ]:
vault write kmip/config listen_addrs=0.0.0.0:5696 \
      tls_ca_key_type="rsa" \
      tls_ca_key_bits=2048

**Retrieve the generated CA certificate**

In [ ]:
vault read kmip/ca

---
### Demo:
**Note**:  
* Run > Render All Markdown Cells
* Edit > Clear all outputs

**Use scopes to partition KMIP managed object storage into multiple named buckets.** 
Within a scope, roles can be created which dictate a set of allowed operations.  TLS client certificates can be generated for a role, which services and applications can then use when sending KMIP requests against Vault's KMIP secret engine.

In [ ]:
vault write -f kmip/scope/finance

In [ ]:
vault list kmip/scope/

In [ ]:
vault write kmip/scope/finance/role/accounting operation_all=true

In [ ]:
vault read kmip/scope/finance/role/accounting

**Generate a client certificate**      
This returns the CA chain, the certificate and the private key.

In [ ]:
vault write -format=json kmip/scope/finance/role/accounting/credential/generate \
  format=pem > /tmp/credential.json

In [ ]:
#openssl x509 -in /tmp/cert.pem -noout -text
jq -r .data.certificate < /tmp/credential.json > /tmp/cert.pem

In [ ]:
jq -r .data.private_key < /tmp/credential.json > /tmp/key.pem

**Note:** the resulting cert.pem and key.pem is what KMIP client will use to establish a connection with Vault KMIP server

In [ ]:
vault list kmip/scope/finance/role/accounting/credential

**Use serial number to lookup the client certificate**

In [ ]:
 vault read kmip/scope/finance/role/accounting/credential/lookup \
        serial_number=5811251589049531701235691672181269233377583790

---
### Demo: Your First Secret
Now that the Vault server is up and running, let's get read and write your first secret.  One of the core features of Vault is the ability to read and write arbitrary secrets securely.  Secrets are written to Vaul are encrypted and then written to backend storage.

Let's write a secret to Key/Value secret engine.  
   
**Note**:  
* Run > Render All Markdown Cells
* Edit > Clear all outputs

### Client Certificate Signing
KMIP secrets engine supports signing of CSR (certificate signing requests).   Some KMIP clients prefer (or only support) retaining the private key associated with their client certificate.

In [ ]:
vault write kmip/scope/my-service/role/admin/credential/sign csr="$(cat /tmp/my-csr.pem)"


### Supported KMIP Operations
Vault policies do not come into play during these KMIP operations.  The KMIP secrets engine determines the set of KMIP operations the clients are allowed to performed based on the roles that are applied to the TLS client certificate.
* operation_activate
* operation_add_attribute
* operation_create
* operation_destroy
* operation_discover_versions
* operation_get
* operation_get_attributes
* operation_get_attribute_list
* operation_locate
* operation_register
* operation_rekey
* operation_revoke

In [ ]:
### Generate root certificate

In [ ]:
vault write pki/root/generate/internal \
   common_name=my-website.com \
   ttl=8760h

**Note** the returned certificate is purely informational.  It and its private key are stored in Vault.
We recommend maintaining your root CA outside of Vault and providing Vault a signed intermediate CA.


### Set URL configuration
Generated certificates can have the CRL (Certifiate Revocation List), a list of digital certificates that have been revoked by the issuing CA, location and the location of the issuing certificate encoded.  These values must be set manually and typically to FQDN associated to the Vault server.  
These can be updated in the future

In [ ]:
vault write pki/config/urls \
   issuing_certificates="http://127.0.0.1:8200/v1/pki/ca" \
   crl_distribution_points="http://127.0.0.1:8200/v1/pki/crl"


### Configure a role 
A role maps a name in Vault to a procedure for generating certificate

In [ ]:
vault write pki/roles/example-dot-com \
   allowed_domains=my-website.com \
   allow_subdomains=true \
   max_ttl=72h

### Generate a new credential

After the secrets engine is configure and a user/machine has a Vault token, it can generae credentials.

In [ ]:
vault write pki/issue/example-dot-com common_name=www.my-website.com

---
### Setting up Intermediate CA
Mount the backend

In [ ]:
vault secrets enable -path=pki_int pki

### Configure an Intermediate CA

In [ ]:
vault secrets tune -max-lease-ttl=43800h pki_int

### Generate intermediate certificate signing request

In [ ]:
vault write pki_int/intermediate/generate/internal common_name="myvault.com Intermediate Authority" \
   ttl=43800h
   

In [ ]:
echo "-----BEGIN CERTIFICATE REQUEST-----
MIICcjCCAVoCAQAwLTErMCkGA1UEAxMibXl2YXVsdC5jb20gSW50ZXJtZWRpYXRl
IEF1dGhvcml0eTCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAL42hybF
uK/RSK2SdhHOwDWxgChJKxnmug+fEBPdrvhg+D3uXWZluXFzAwnnFtgPkK+GmXg0
ZXmTsnx30CtHksADh7GmkrA7PU8f/NdCiPXGrxsibGYD7aNRE8Y7MUIorcYPsY3d
zQ+/a15oM//GQRoMXPL68xnnCnPBP63DC+2YQzr0szQJIuEcoUYFjvI/0f8HMTEr
HfcUP3CMmRxEFLu0UloppK3UGXc9ku7r8BsOzovVOCvC6bzVg+lgBsIIV+2w97IM
VGl26Mu0WxVk+ECkK9Lheb4PX53kZ693+JumpNMQFMLF3FlBqtCO7Cni4HtG/xgo
QcG7dTNeDnLNmJsCAwEAAaAAMA0GCSqGSIb3DQEBCwUAA4IBAQAZcen7BXllqG7O
q87395bLdDA1fPTjb5AROMUuCUsJUZKP9dX4rdPD1q0ASijFLRLWtDNmX1kW9o70
YCX/pF0qNErIu4GejDAHACY4jTQL7hrVIpL5QPPCILF9TSxJLkZgEQfzTnyN0Unn
E1BhKO9NmcNNzE3gRgWlsqOL6Rvd9gr9bAcUQNFWRL13nFFY0JMXocgR6V/t8hUh
gB21kS0X/NqKO8el5DrrTKQjKSsnqQ3JjSLF2YgC1/w6rVjhfD9IIWqaY58KqPZv
BxutThLBiDFdWIpt0SNky3tYxco4dbVwDABr31CoemsE0Iooln4/o5nrHpMMJS0+
hZg20o1D
-----END CERTIFICATE REQUEST-----
" > /tmp/pki_int.csr
cat /tmp/pki_int.csr


### Sign CSR using another certificate authority, 
in this case the root CA generated in the first example

In [ ]:
vault write pki/root/sign-intermediate csr=@/tmp/pki_int.csr format=pem_bundle ttl=32800h

&nbsp;

Now set the intermediate certificate authorities signing certificate to the root-signed certificate

In [ ]:
vault write pki_int/intermediate/set-signed certificate=@signed_certificate.pem

### Set URL configuration
Generate certificates can have the CRL location and the location of the issuing certificate encoded.

In [ ]:
vault write pki_int/config/urls \
  issuing_certificates="http://127.0.0.1:8200/v1/pki_int/ca" \
  crl_distribution_points="http://127.0.0.1:8200/v1/pki_int/crl"

### Configure a role

In [ ]:
vault write pki_int/roles/example-dot-com \
   allowed_domains=example.com \
   allow_subdomain=true max_ttl=72h

### Issue Certificates

In [ ]:
vault write pki_int/issue/example-dot-com common_name=blah.example.com

&nbsp;

Vault now has generated a new set of credentials usign the example-dot-com role coniguration.  Here we see the dynamically generated private key and certificate.  The issuing CA certificate and CA trust chain are returned as well.
The CA Chain returns all the intermeiate authorities in the trust chain.  The root authority is not included since that will usually be trusted by the underlying OS.

---
### Use web UI

Open a browser and visit http://127.0.0.1:8200


&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">